In [43]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime
import re
import time
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_squared_log_error, confusion_matrix
from scipy.stats import uniform, randint
from sklearn.model_selection import TimeSeriesSplit, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import math
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

In [44]:
# data upload
training_data = pd.read_csv(r'C:\Users\Elizabeth\Desktop\data_science\team_ruth\data\TrainingSet.csv',index_col=0)
submission_labels = pd.read_csv(r'C:\Users\Elizabeth\Desktop\data_science\team_ruth\data\SubmissionRows.csv',index_col=0)

In [45]:
training_data.head(2)

,1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],Country Name,Series Code,Series Name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.769214,Afghanistan,allsi.bi_q1,(%) Benefits held by 1st 20% population - All Social Insurance
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.027746,Afghanistan,allsp.bi_q1,(%) Benefits held by 1st 20% population - All Social Protection


In [46]:
# fuction to split that dataframe to values [2005,2006,2007] and 
def split_dataframe(data):
    raw_data = data.loc[:,:'2007 [YR2007]']
    #raw_data=data.iloc[:,:-1] 
    description = data.loc[:,'Country Name':]
    #description=data.iloc[:,-1] 
    return raw_data,description

raw_data, description = split_dataframe(training_data)
description.head()

,Country Name,Series Code,Series Name
0,Afghanistan,allsi.bi_q1,(%) Benefits held by 1st 20% population - All Social Insurance
1,Afghanistan,allsp.bi_q1,(%) Benefits held by 1st 20% population - All Social Protection
2,Afghanistan,allsa.bi_q1,(%) Benefits held by 1st 20% population - All Social Safety Nets
4,Afghanistan,allsi.gen_pop,(%) Generosity of All Social Insurance
5,Afghanistan,allsp.gen_pop,(%) Generosity of All Social Protection


In [47]:
# impute the missing values using interpolation
raw_data = raw_data.interpolate(limit_direction='both',axis=1)
raw_data.head()

,1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007]
0,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214
1,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746
2,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887
4,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105
5,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814


In [48]:
training_data=raw_data.join(description)
training_data.head(2)

,1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],Country Name,Series Code,Series Name
0,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,Afghanistan,allsi.bi_q1,(%) Benefits held by 1st 20% population - All Social Insurance
1,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,Afghanistan,allsp.bi_q1,(%) Benefits held by 1st 20% population - All Social Protection


In [49]:
training_pivot = training_data.melt(id_vars=['Series Code', 'Country Name'], var_name='Year', value_name='indicator',ignore_index = False, \
                        value_vars=['1972 [YR1972]','1973 [YR1973]','1974 [YR1974]','1975 [YR1975]',\
                                    '1976 [YR1976]','1977 [YR1977]','1978 [YR1978]','1979 [YR1979]', \
                                    '1980 [YR1980]',  '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]', \
                                    '1984 [YR1984]','1985 [YR1985]','1986 [YR1986]','1987 [YR1987]', \
                                    '1988 [YR1988]','1989 [YR1989]','1990 [YR1990]','1991 [YR1991]', \
                                    '1992 [YR1992]','1993 [YR1993]','1994 [YR1994]','1995 [YR1995]', \
                                    '1996 [YR1996]', '1997 [YR1997]','1998 [YR1998]','1999 [YR1999]', \
                                    '2000 [YR2000]','2001 [YR2001]','2002 [YR2002]','2003 [YR2003]' , \
                                    '2004 [YR2004]','2005 [YR2005]','2006 [YR2006]','2007 [YR2007]'])

In [50]:
training_pivot['Year'] = training_pivot['Year'].str.extract('(\d+)', expand=False) #.4stype(i4t),'2005 [YR2005]','2006 [YR2006]'
pd.set_option('display.max_colwidth', None)
training_pivot['Year'] = pd.to_datetime(training_pivot['Year'], format='%Y')

In [51]:
imputed_results_raw = training_pivot

# Only fetch the row required for prediction as presence of other indicators causes data skew problems 
imputed_results = imputed_results_raw.join(submission_labels,how='inner')
imputed_results = imputed_results.rename({'Series_Code':'Series Code','Country_Name': 'Country Name'},axis=1)
imputed_results = imputed_results.drop(['2008 [YR2008]','2012 [YR2012]'], axis=1)

In [52]:
print("min:: " + str(imputed_results["indicator"].min()) + "\nmax:: " + str(imputed_results["indicator"].max()))

min:: 0.0
max:: 1.0


In [53]:
# ensure null values are imputed 
df=imputed_results[imputed_results.indicator.isnull()]
df.head()

,Series Code,Country Name,Year,indicator


In [54]:
#Select the number of years to compute for
imputed_results=imputed_results[pd.to_datetime(imputed_results['Year'], format='%Y-%m-%d') >= '1990-01-01']
imputed_results.head()

,Series Code,Country Name,Year,indicator
559,7.8,Afghanistan,1990-01-01,0.048
559,7.8,Afghanistan,1991-01-01,0.048
559,7.8,Afghanistan,1992-01-01,0.049
559,7.8,Afghanistan,1993-01-01,0.049
559,7.8,Afghanistan,1994-01-01,0.049


__Data Preprocessing__

In [55]:
# Append the continent and sub region 
countryContinent= pd.read_csv('https://raw.githubusercontent.com/mansim07/udgm/main/data/countryContinentutf8.csv')
countryContinent=countryContinent.fillna(0)
countryContinent=countryContinent.rename({'country':'Country Name'},axis=1)
countryContinent[countryContinent.region_code.isnull()]

,Country Name,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code


In [56]:
imputed_results= pd.merge(left=imputed_results, right=countryContinent, how='left', left_on='Country Name', right_on='Country Name')
imputed_results.count()

Series Code        13266
Country Name       13266
Year               13266
indicator          13266
code_2             13194
code_3             13194
country_code       13194
iso_3166_2         13194
continent          13194
sub_region         13194
region_code        13194
sub_region_code    13194
dtype: int64

In [57]:
#Ensure null values are imputed 
imputed_results=imputed_results.fillna(0)
df=imputed_results[imputed_results.region_code.isnull()]
df.groupby("Country Name").nunique()

,Series Code,Year,indicator,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
Country Name,,,,,,,,,,,


In [58]:
# Apply label encoding of categorical features
encode_columns = ['Country Name','Series Code']
encode_df = imputed_results[encode_columns]
encode_df = encode_df.astype('str')
encode_df = encode_df.apply(LabelEncoder().fit_transform)
score_encode_drop = imputed_results.drop(encode_columns, axis = 1)

score_encode = pd.concat([score_encode_drop, encode_df], axis = 1)
score_encode['Year']=pd.to_datetime(imputed_results['Year'], format='%Y-%m-%d').dt.year
score_target=score_encode
score_target.head()

,Year,indicator,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code,Country Name,Series Code
0,1990,0.048,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5
1,1991,0.048,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5
2,1992,0.049,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5
3,1993,0.049,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5
4,1994,0.049,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5


In [59]:
# split data to training and testing sets
split_date = 2006
train_df = score_target.loc[score_target['Year'] <= split_date].copy()
test_df = score_target.loc[score_target['Year'] > split_date].copy()

def create_features(df, label=None):
    X = df[['country_code','Series Code','region_code','sub_region_code', 'Year']] 
    
    if label:
        y = df[label]
        return X, y
    return X

X_train, y_train = create_features(train_df, label='indicator')
X_test, y_test = create_features(test_df, label='indicator')
X_train.head()

,country_code,Series Code,region_code,sub_region_code,Year
0,4.0,5,142.0,34.0,1990
1,4.0,5,142.0,34.0,1991
2,4.0,5,142.0,34.0,1992
3,4.0,5,142.0,34.0,1993
4,4.0,5,142.0,34.0,1994


In [74]:
# Simple Regression Model 
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))
print ('LinearRegression RMSE:', rmse)

LinearRegression RMSE: 0.38943286100512803


__Random Forest__

In [75]:
#Random forest Model
model = RandomForestRegressor(n_estimators=1000,
                              random_state=123,
                              max_features=5,
                              max_depth=100,
                              min_samples_split=2,
                              min_samples_leaf=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, y_pred))

print ('RandomForest RMSE:', rmse)

RandomForest RMSE: 0.028594541802297636


In [78]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [500, 1000],
    'max_depth': [10, 20],
    'max_features': [2, 10],
    'min_samples_leaf': [3, 5],
    'min_samples_split': [5, 10, 20]
}
# create based model
rf = RandomForestRegressor(random_state=42)
# instantantiate grid search model
grid_search = GridSearchCV(estimator=rf,
                                param_grid=param_grid, 
                                cv=5, 
                                verbose=2, 
                                n_jobs=-1)

grid_search.fit(X_train,y_train)
print(grid_search.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.90624446 0.90733748 0.90616318 0.90659457 0.90266227 0.90255516
 0.90394827 0.90493702 0.90394827 0.90493702 0.90300158 0.90280799
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.90818315 0.9085733  0.90869521 0.90934327 0.90809947 0.90759754
 0.90954662 0.9098139  0.90954662 0.9098139  0.9079902  0.90785916
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan]
  warnings.warn(


{'max_depth': 20, 'max_features': 2, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 1000}


In [80]:
# random forest with best params
model = RandomForestRegressor(n_estimators=1000,
                              max_features=2,
                              max_depth=20,
                              min_samples_split=5,
                              min_samples_leaf=5)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, y_pred))

print ('RandomForest RMSE:', rmse)

RandomForest RMSE: 0.07821687632066458


__2008 Predictions__

In [62]:
submission_results=submission_labels.join(training_data)
submission_results=submission_results[['Country Name','Series Code']]
submission_results['Year']=2008
submission_results['indicator']=0

# label encoding
encode_columns = ['Series Code']
pred_df = submission_results[encode_columns]
score_predict_drop = submission_results.drop(encode_columns, axis = 1)
pred_df = pred_df.astype('str')
pred_df = pred_df.apply(LabelEncoder().fit_transform)


score_predict_2008 = pd.concat([score_predict_drop, pred_df], axis = 1)
score_predict_2008['Year']=pd.to_datetime(score_predict_2008['Year'],format='%Y').dt.year
score_predict_2008=pd.merge(left=score_predict_2008, right=countryContinent, how='left', left_on='Country Name', right_on='Country Name').set_axis(score_predict_2008.index)
score_predict_2008=score_predict_2008.fillna(0)


#Replace the model name here for different models 
# random forest
rf08_pred = rf.predict(score_predict_2008[['country_code','Series Code','region_code','sub_region_code', 'Year']])

score_predict_2008['predictions'] = rf08_pred

score_predict_2008.head()

,Country Name,Year,indicator,Series Code,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code,predictions
559,Afghanistan,2008,0,5,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.410667
618,Afghanistan,2008,0,6,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.026489
753,Afghanistan,2008,0,1,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.116347
1030,Afghanistan,2008,0,3,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.001057
1896,Albania,2008,0,5,AL,ALB,8.0,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0,0.962259


__2012 Predictions__

In [63]:
submission_results = submission_labels.join(training_data)
submission_results=submission_results[['Country Name','Series Code']]
submission_results['Year']=2012
submission_results['indicator']=0

# Label Encoding 
encode_columns = ['Series Code']
pred_df = submission_results[encode_columns]
score_predict_drop = submission_results.drop(encode_columns, axis = 1)
pred_df = pred_df.astype('str')
pred_df = pred_df.apply(LabelEncoder().fit_transform)

score_predict_2012 = pd.concat([score_predict_drop, pred_df], axis = 1)
score_predict_2012['Year']=pd.to_datetime(score_predict_2012['Year'],format='%Y').dt.year
score_predict_2012=pd.merge(left=score_predict_2012, right=countryContinent, how='left', left_on='Country Name', right_on='Country Name').set_axis(score_predict_2012.index)
score_predict_2012=score_predict_2012.fillna(0)

#Replace the model name here for different models 
# random forest
rf12_pred = model.predict(score_predict_2012[['country_code','Series Code','region_code','sub_region_code', 'Year']])
score_predict_2012['predictions'] = rf12_pred
score_predict_2012.head()

,Country Name,Year,indicator,Series Code,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code,predictions
559,Afghanistan,2012,0,5,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.410667
618,Afghanistan,2012,0,6,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.026489
753,Afghanistan,2012,0,1,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.116347
1030,Afghanistan,2012,0,3,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.001057
1896,Albania,2012,0,5,AL,ALB,8.0,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0,0.962259


In [37]:
predictions_from_model=score_predict_2012.join(score_predict_2008,how='inner',lsuffix=2012, rsuffix=2008)
predictions_from_model=predictions_from_model.drop(columns=['Year2012','indicator2012','Country Name2012','Year2008','indicator2008','Country Name2008','Series Code2012','Series Code2008'])
predictions_from_model=predictions_from_model.rename(columns={"predictions2008": "2008 [YR2008]", "predictions2012": "2012 [YR2012]"})
predictions_from_model=predictions_from_model[['2008 [YR2008]','2012 [YR2012]']]
predictions_from_model.to_csv("./data/result_randomforest_0120_05.csv")

# Deep Learning

In [82]:
!pip install keras

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)


In [83]:
import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [92]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(5, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='sgd')
    return model

In [93]:
# evaluate model
model = KerasRegressor(build_fn=baseline_model, verbose=0)

# define grid search parameters
epochs = [10, 20]
batch_size = [10, 20]  
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means=grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\tensorflow\python\keras\wrappers\scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\tensorflow\python\eager\def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\Elizabeth\anacond

ValueError: in user code:

    C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Elizabeth\anaconda3\envs\lyt\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Elizabeth\anaconda3\envs\lyt\l